In [1]:
%load_ext dotenv
%dotenv

In [2]:
import json
import os
import requests
from web3 import Web3

In [3]:
w3 = Web3(Web3.HTTPProvider(os.environ['GOERLI_RPC']))
w3.isConnected()

True

In [4]:
CONTRACT_POOL = "0xAf593430b86a0560818a9dF5858B14dDC469Ab98";
CONTRACT_dNFT = "0x0eDb1483320DbdA0C804C1F5673F1733382e37E7";

In [5]:
pool = w3.eth.contract(address=CONTRACT_POOL, abi=json.load(open("abi/Pool.json"))["abi"])
dnft = w3.eth.contract(address=CONTRACT_dNFT, abi=json.load(open("abi/dNFT.json"))["abi"])

In [7]:
INFURA_KEY = os.environ["ETHERSCAN_API"]
url = f"https://api-goerli.etherscan.io/api?module=account&action=txlist&address={CONTRACT_POOL}&startblock=0&endblock=99999999&sort=asc&apikey={INFURA_KEY}"
url

'https://api-goerli.etherscan.io/api?module=account&action=txlist&address=0xAf593430b86a0560818a9dF5858B14dDC469Ab98&startblock=0&endblock=99999999&sort=asc&apikey=TECYJEKT36HQRFI9FA5GESVDQTQGUKJZPX'

In [8]:
r = requests.get(url=url)
txs = r.json()['result']

In [9]:
blockNumbers = []
for tx in txs:
    if tx["functionName"] == "sync()":
        blockNumbers.append(int(tx["blockNumber"]))
blockNumbers

[8095106, 8095157, 8099237, 8099246, 8099251]

In [ ]:
nfts = []

for blockNumber in blockNumbers:
    print(f"block: {blockNumber} / {len}")
    numberOfMints = dnft.functions.numberOfMints().call(block_identifier=blockNumber);
    for i in range(numberOfMints):
        if i % 10 == 0: print(f"nft: {i} / {numberOfMints}")
        tokenId = dnft.functions.tokenByIndex(i) .call(block_identifier=blockNumber)
        nft     = dnft.functions.idToNft(tokenId).call()

        nfts.append({"tokenId": tokenId, 
                     "blockNumber": blockNumber, 
                     "withdrawn":nft[0],
                     "deposit":nft[1],
                     "xp":nft[2]})
nfts